In [12]:
import pubchempy as pcp
import pandas as pd

df = pd.read_csv('data/r4n_c20_validated_cid.csv')
df = df.drop(columns=['name','molecular_weight'])
df.head()

,Index,Num_c,SMILES,cid
0,1,4,C[N+](C)(C)C,6380.0
1,2,5,CC[N+](C)(C)C,33993.0
2,3,6,CCC[N+](C)(C)C,29385.0
3,4,6,CC[N+](C)(C)CC,199517.0
4,5,7,CCCC[N+](C)(C)C,24399.0


In [13]:
def get_ion_properties(cid_list, properties=None):
    """
    使用 get_properties 批量获取离子的性质
    
    参数:
        cid_list: CID列表
        properties: 要获取的属性列表,如果为None则获取常用属性
    
    返回:
        包含所有化合物信息的DataFrame
    """
    if properties is None:
        # 默认获取的属性 - 使用正确的属性名称
        properties = [
            'MolecularFormula',
            'MolecularWeight',
            'IUPACName',
            'Charge',
            'Complexity',
            'HBondDonorCount',
            'HBondAcceptorCount',
            'RotatableBondCount',
            'HeavyAtomCount',
            'TPSA',  # 拓扑极性表面积
            'XLogP',  # 分配系数
            'ExactMass',
            'MonoisotopicMass',
            # 3D 属性
            'Volume3D',
            'XStericQuadrupole3D',
            'YStericQuadrupole3D',
            'ZStericQuadrupole3D',
            'FeatureCount3D',
            'FeatureAcceptorCount3D',
            'FeatureDonorCount3D',
            'FeatureAnionCount3D',
            'FeatureCationCount3D',
            'FeatureRingCount3D',
            'FeatureHydrophobeCount3D',
            'ConformerModelRMSD3D',
            'EffectiveRotorCount3D',
            'ConformerCount3D',
        ]
    
    try:
        # 批量获取属性
        results = pcp.get_properties(properties, cid_list, namespace='cid')
        
        # 转换为DataFrame
        df_results = pd.DataFrame(results)
        
        return df_results
    
    except Exception as e:
        print(f"Error: {e}")
        return None

In [14]:
# 示例: 批量处理所有CID (这里只演示前20个以节省时间)
print(f"There are {len(df)} R4N+ ions")

# 获取前20个CID
cid_list = df['cid'].astype(int).tolist()

# 批量获取属性
print(f"Searching {len(cid_list)} compounds for properties...")
result_all = get_ion_properties(cid_list)

if result_all is not None:
    # 转换数值列为正确的类型
    numeric_cols = ['MolecularWeight', 'Volume3D', 'EffectiveRotorCount3D', 
                    'Charge', 'TPSA', 'XLogP', 'Complexity']
    for col in numeric_cols:
        if col in result_all.columns:
            result_all[col] = pd.to_numeric(result_all[col], errors='coerce')

    print(f"Successfully retrieved properties for {len(result_all)} compounds\n")

There are 235 R4N+ ions
Searching 235 compounds for properties...
Successfully retrieved properties for 235 compounds

Successfully retrieved properties for 235 compounds



In [15]:
# 把获取到的性质合并到原始DataFrame
df = df.merge(result_all, left_on='cid', right_on='CID', how='left')
df.head()

,Index,Num_c,SMILES,cid,CID,MolecularFormula,MolecularWeight,IUPACName,XLogP,ExactMass,...,FeatureCount3D,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D
0,1,4,C[N+](C)(C)C,6380.0,6380,C4H12N+,74.14,tetramethylazanium,0.3,74.096974387,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.4,0.0,1
1,2,5,CC[N+](C)(C)C,33993.0,33993,C5H14N+,88.17,ethyl(trimethyl)azanium,0.6,88.112624451,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.4,1.0,1
2,3,6,CCC[N+](C)(C)C,29385.0,29385,C6H16N+,102.20,trimethyl(propyl)azanium,1.2,102.128274515,...,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.4,2.0,1
3,4,6,CC[N+](C)(C)CC,199517.0,199517,C6H16N+,102.20,diethyl(dimethyl)azanium,1.0,102.128274515,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.4,2.0,5
4,5,7,CCCC[N+](C)(C)C,24399.0,24399,C7H18N+,116.22,butyl(trimethyl)azanium,1.5,116.143924578,...,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.6,3.0,3


In [ ]:
df_to_save = df.drop(columns=['cid'])
df_to_save.to_csv('data/r4n_c20_validated_pubchempy.csv', index=False)

## 关键性质筛选 - 用于预测吸附能和水溶性

基于季铵阳离子在金属表面吸附和水溶性的理化机制,选择以下关键性质:

In [16]:
# 定义关键性质及其对吸附能和水溶性的影响

key_properties = {
    # === 分子大小和形状 (影响吸附位点和表面覆盖) ===
    'MolecularWeight': '分子量 - 影响扩散和吸附动力学',
    'Volume3D': '3D体积 - 决定表面覆盖率和空间排布',
    'HeavyAtomCount': '重原子数 - 反映分子大小',
    
    # === 电荷和极性 (直接影响静电吸附) ===
    'Charge': '电荷 - 与金属表面静电相互作用的关键(阳离子+1)',
    'TPSA': '拓扑极性表面积 - 影响水溶性和极性相互作用',
    
    # === 疏水性 (影响在水相中的行为和界面吸附) ===
    'XLogP': '分配系数 - 疏水性指标,影响水溶性和界面吸附倾向',
    
    # === 分子复杂度和结构 ===
    'Complexity': '分子复杂度 - 反映结构特征',
    'RotatableBondCount': '可旋转键数 - 分子柔性,影响构象适应性',
    'EffectiveRotorCount3D': '有效转子数(3D) - 实际构象灵活性',
    
    # === 氢键能力 (影响与水和表面的相互作用) ===
    'HBondDonorCount': '氢键供体数 - 与水和表面OH基团的相互作用',
    'HBondAcceptorCount': '氢键受体数 - 影响溶剂化和表面作用',
    
    # === 3D立体特征 (影响表面吸附几何) ===
    'XStericQuadrupole3D': 'X轴立体四极矩 - 电荷分布非对称性',
    'YStericQuadrupole3D': 'Y轴立体四极矩 - 电荷分布非对称性',
    'ZStericQuadrupole3D': 'Z轴立体四极矩 - 电荷分布非对称性',
    
    # === 功能位点 (影响吸附模式) ===
    'FeatureCationCount3D': '阳离子特征数 - 带正电中心(季铵N+)',
    'FeatureHydrophobeCount3D': '疏水特征数 - 烷基链的疏水性',
    
    # === 构象信息 ===
    'ConformerCount3D': '构象数 - 可能的空间构型数量',
    'ConformerModelRMSD3D': '构象RMSD - 构象变化幅度',
}

print("=== 关键性质列表 ===\n")
print(f"共选择 {len(key_properties)} 个关键性质:\n")

for i, (prop, desc) in enumerate(key_properties.items(), 1):
    print(f"{i:2d}. {prop:30s} - {desc}")

=== 关键性质列表 ===

共选择 18 个关键性质:

 1. MolecularWeight                - 分子量 - 影响扩散和吸附动力学
 2. Volume3D                       - 3D体积 - 决定表面覆盖率和空间排布
 3. HeavyAtomCount                 - 重原子数 - 反映分子大小
 4. Charge                         - 电荷 - 与金属表面静电相互作用的关键(阳离子+1)
 5. TPSA                           - 拓扑极性表面积 - 影响水溶性和极性相互作用
 6. XLogP                          - 分配系数 - 疏水性指标,影响水溶性和界面吸附倾向
 7. Complexity                     - 分子复杂度 - 反映结构特征
 8. RotatableBondCount             - 可旋转键数 - 分子柔性,影响构象适应性
 9. EffectiveRotorCount3D          - 有效转子数(3D) - 实际构象灵活性
10. HBondDonorCount                - 氢键供体数 - 与水和表面OH基团的相互作用
11. HBondAcceptorCount             - 氢键受体数 - 影响溶剂化和表面作用
12. XStericQuadrupole3D            - X轴立体四极矩 - 电荷分布非对称性
13. YStericQuadrupole3D            - Y轴立体四极矩 - 电荷分布非对称性
14. ZStericQuadrupole3D            - Z轴立体四极矩 - 电荷分布非对称性
15. FeatureCationCount3D           - 阳离子特征数 - 带正电中心(季铵N+)
16. FeatureHydrophobeCount3D       - 疏水特征数 - 烷基链的疏水性
17. ConformerCount3D               - 构象数 - 可能的空间构型数量
18. C

In [17]:
# 提取关键性质并检查数据完整性
key_props_list = list(key_properties.keys())

# 检查哪些性质在数据中可用
available_props = [prop for prop in key_props_list if prop in df.columns]
missing_props = [prop for prop in key_props_list if prop not in df.columns]

print(f"可用的关键性质: {len(available_props)}/{len(key_props_list)}")
print(f"缺失的性质: {missing_props if missing_props else '无'}\n")

# 创建包含关键性质的DataFrame
df_key_props = df[['Index', 'Num_c', 'SMILES', 'CID', 'IUPACName'] + available_props].copy()

# 检查每个性质的数据完整性
print("数据完整性检查:")
print("="*70)
for prop in available_props:
    non_null = df_key_props[prop].notna().sum()
    null_count = df_key_props[prop].isna().sum()
    mean_val = df_key_props[prop].mean() if pd.api.types.is_numeric_dtype(df_key_props[prop]) else "N/A"
    
    if pd.api.types.is_numeric_dtype(df_key_props[prop]):
        print(f"{prop:30s}: {non_null:3d}/{len(df_key_props)} 非空 (缺失:{null_count:3d}) | 均值: {mean_val:.2f}")
    else:
        print(f"{prop:30s}: {non_null:3d}/{len(df_key_props)} 非空 (缺失:{null_count:3d})")

print(f"\n数据集大小: {len(df_key_props)} 个季铵阳离子")

可用的关键性质: 18/18
缺失的性质: 无

数据完整性检查:
MolecularWeight               : 235/235 非空 (缺失:  0) | 均值: 231.00
Volume3D                      : 211/235 非空 (缺失: 24) | 均值: 205.18
HeavyAtomCount                : 235/235 非空 (缺失:  0) | 均值: 16.18
Charge                        : 235/235 非空 (缺失:  0) | 均值: 1.00
TPSA                          : 235/235 非空 (缺失:  0) | 均值: 0.00
XLogP                         : 235/235 非空 (缺失:  0) | 均值: 5.48
Complexity                    : 235/235 非空 (缺失:  0) | 均值: 137.79
RotatableBondCount            : 235/235 非空 (缺失:  0) | 均值: 11.18
EffectiveRotorCount3D         : 211/235 非空 (缺失: 24) | 均值: 10.64
HBondDonorCount               : 235/235 非空 (缺失:  0) | 均值: 0.00
HBondAcceptorCount            : 235/235 非空 (缺失:  0) | 均值: 0.00
XStericQuadrupole3D           : 211/235 非空 (缺失: 24) | 均值: 14.21
YStericQuadrupole3D           : 211/235 非空 (缺失: 24) | 均值: 3.01
ZStericQuadrupole3D           : 211/235 非空 (缺失: 24) | 均值: 1.32
FeatureCationCount3D          : 211/235 非空 (缺失: 24) | 均值: 1.00
FeatureHydr

In [18]:
# 统计分析关键性质
import numpy as np

print("\n=== 关键性质统计分析 ===\n")

# 选择数值型性质进行统计
numeric_props = [prop for prop in available_props 
                 if pd.api.types.is_numeric_dtype(df_key_props[prop])]

stats_df = df_key_props[numeric_props].describe().T
stats_df['range'] = stats_df['max'] - stats_df['min']
stats_df['cv'] = (stats_df['std'] / stats_df['mean']) * 100  # 变异系数

print("统计摘要 (关键数值性质):")
print("="*100)
print(stats_df[['mean', 'std', 'min', 'max', 'range', 'cv']].round(2).to_string())

print("\n\n变异系数(CV)解读:")
print("  - CV < 10%: 变化很小")
print("  - CV 10-30%: 中等变化")
print("  - CV > 30%: 变化较大,可能是重要的区分特征")


=== 关键性质统计分析 ===

统计摘要 (关键数值性质):
                            mean    std    min     max   range     cv
MolecularWeight           231.00  50.15  74.14  298.60  224.46  21.71
Volume3D                  205.18  43.36  70.80  265.10  194.30  21.13
HeavyAtomCount             16.18   3.58   5.00   21.00   16.00  22.09
Charge                      1.00   0.00   1.00    1.00    0.00   0.00
TPSA                        0.00   0.00   0.00    0.00    0.00    NaN
XLogP                       5.48   1.81   0.30    8.40    8.10  33.04
Complexity                137.79  39.35  19.00  202.00  183.00  28.56
RotatableBondCount         11.18   3.58   0.00   16.00   16.00  31.97
EffectiveRotorCount3D      10.64   3.36   0.00   15.00   15.00  31.59
HBondDonorCount             0.00   0.00   0.00    0.00    0.00    NaN
HBondAcceptorCount          0.00   0.00   0.00    0.00    0.00    NaN
XStericQuadrupole3D        14.21   9.57   1.25   47.87   46.62  67.37
YStericQuadrupole3D         3.01   1.61   0.91    8.85  

### 特征重要性分析

基于物理化学机制,对关键性质进行分类和重要性排序:

In [19]:
# 按机制分类特征
feature_categories = {
    '🔋 吸附能关键特征': {
        'props': ['Charge', 'Volume3D', 'XLogP', 'FeatureCationCount3D', 
                  'XStericQuadrupole3D', 'YStericQuadrupole3D', 'ZStericQuadrupole3D'],
        'mechanism': '静电作用 + 疏水作用 + 空间匹配'
    },
    
    '💧 水溶性关键特征': {
        'props': ['XLogP', 'TPSA', 'HBondDonorCount', 'HBondAcceptorCount', 
                  'MolecularWeight'],
        'mechanism': '疏水/亲水平衡 + 氢键作用'
    },
    
    '📐 结构灵活性': {
        'props': ['RotatableBondCount', 'EffectiveRotorCount3D', 'ConformerCount3D', 
                  'ConformerModelRMSD3D'],
        'mechanism': '构象适应性 + 表面构象调整'
    },
    
    '🔬 分子大小': {
        'props': ['MolecularWeight', 'Volume3D', 'HeavyAtomCount'],
        'mechanism': '表面覆盖率 + 扩散动力学'
    },
    
    '⚡ 疏水特征': {
        'props': ['FeatureHydrophobeCount3D', 'XLogP'],
        'mechanism': '烷基链疏水作用'
    }
}

print("=== 按机制分类的关键特征 ===\n")
for category, info in feature_categories.items():
    print(f"\n{category}")
    print(f"机制: {info['mechanism']}")
    print("性质:")
    for prop in info['props']:
        if prop in available_props:
            status = "✓"
            if pd.api.types.is_numeric_dtype(df_key_props[prop]):
                mean_val = df_key_props[prop].mean()
                std_val = df_key_props[prop].std()
                print(f"  {status} {prop:30s} (均值: {mean_val:7.2f} ± {std_val:6.2f})")
            else:
                print(f"  {status} {prop:30s}")
        else:
            print(f"  ✗ {prop:30s} [缺失]")
    print("-" * 80)

=== 按机制分类的关键特征 ===


🔋 吸附能关键特征
机制: 静电作用 + 疏水作用 + 空间匹配
性质:
  ✓ Charge                         (均值:    1.00 ±   0.00)
  ✓ Volume3D                       (均值:  205.18 ±  43.36)
  ✓ XLogP                          (均值:    5.48 ±   1.81)
  ✓ FeatureCationCount3D           (均值:    1.00 ±   0.00)
  ✓ XStericQuadrupole3D            (均值:   14.21 ±   9.57)
  ✓ YStericQuadrupole3D            (均值:    3.01 ±   1.61)
  ✓ ZStericQuadrupole3D            (均值:    1.32 ±   0.59)
--------------------------------------------------------------------------------

💧 水溶性关键特征
机制: 疏水/亲水平衡 + 氢键作用
性质:
  ✓ XLogP                          (均值:    5.48 ±   1.81)
  ✓ TPSA                           (均值:    0.00 ±   0.00)
  ✓ HBondDonorCount                (均值:    0.00 ±   0.00)
  ✓ HBondAcceptorCount             (均值:    0.00 ±   0.00)
  ✓ MolecularWeight                (均值:  231.00 ±  50.15)
--------------------------------------------------------------------------------

📐 结构灵活性
机制: 构象适应性 + 表面构象调整
性质:
  ✓ RotatableBondC

### 推荐的核心特征集

根据物理化学机制和实际应用,推荐以下核心特征用于建模:

In [20]:
# 推荐的核心特征集
core_features = {
    'Tier 1 - 最关键特征 (直接影响吸附)': [
        'XLogP',              # 疏水性 - 影响界面吸附倾向
        'Volume3D',           # 分子体积 - 空间排布和覆盖
        'MolecularWeight',    # 分子量 - 大小效应
        'Charge',             # 电荷 - 静电相互作用(全部+1)
    ],
    
    'Tier 2 - 重要特征 (影响构象和溶解性)': [
        'EffectiveRotorCount3D',    # 有效转子数 - 柔性
        'RotatableBondCount',       # 可旋转键 - 构象灵活性
        'TPSA',                     # 极性表面积 - 水溶性
        'Complexity',               # 分子复杂度
    ],
    
    'Tier 3 - 辅助特征 (细节和结构信息)': [
        'HeavyAtomCount',           # 重原子数
        'FeatureCationCount3D',     # 阳离子特征
        'FeatureHydrophobeCount3D', # 疏水特征
        'XStericQuadrupole3D',      # 立体四极矩X
        'YStericQuadrupole3D',      # 立体四极矩Y
        'ZStericQuadrupole3D',      # 立体四极矩Z
        'ConformerCount3D',         # 构象数
    ]
}

print("=== 推荐的核心特征集 ===\n")

all_core_features = []
for tier, features in core_features.items():
    print(f"\n{tier}")
    print("=" * 80)
    for feat in features:
        all_core_features.append(feat)
        if feat in df_key_props.columns:
            if pd.api.types.is_numeric_dtype(df_key_props[feat]):
                mean_val = df_key_props[feat].mean()
                std_val = df_key_props[feat].std()
                min_val = df_key_props[feat].min()
                max_val = df_key_props[feat].max()
                print(f"  ✓ {feat:30s} | {mean_val:7.2f} ± {std_val:6.2f} | 范围: [{min_val:7.2f}, {max_val:7.2f}]")
            else:
                print(f"  ✓ {feat:30s}")
        else:
            print(f"  ✗ {feat:30s} [缺失]")

print(f"\n\n总计推荐 {len(all_core_features)} 个核心特征")

=== 推荐的核心特征集 ===


Tier 1 - 最关键特征 (直接影响吸附)
  ✓ XLogP                          |    5.48 ±   1.81 | 范围: [   0.30,    8.40]
  ✓ Volume3D                       |  205.18 ±  43.36 | 范围: [  70.80,  265.10]
  ✓ MolecularWeight                |  231.00 ±  50.15 | 范围: [  74.14,  298.60]
  ✓ Charge                         |    1.00 ±   0.00 | 范围: [   1.00,    1.00]

Tier 2 - 重要特征 (影响构象和溶解性)
  ✓ EffectiveRotorCount3D          |   10.64 ±   3.36 | 范围: [   0.00,   15.00]
  ✓ RotatableBondCount             |   11.18 ±   3.58 | 范围: [   0.00,   16.00]
  ✓ TPSA                           |    0.00 ±   0.00 | 范围: [   0.00,    0.00]
  ✓ Complexity                     |  137.79 ±  39.35 | 范围: [  19.00,  202.00]

Tier 3 - 辅助特征 (细节和结构信息)
  ✓ HeavyAtomCount                 |   16.18 ±   3.58 | 范围: [   5.00,   21.00]
  ✓ FeatureCationCount3D           |    1.00 ±   0.00 | 范围: [   1.00,    1.00]
  ✓ FeatureHydrophobeCount3D       |    2.87 ±   1.44 | 范围: [   0.00,    6.00]
  ✓ XStericQuadrupole3D            | 

In [21]:
# 创建包含核心特征的精简数据集
df_ml_ready = df_key_props[['Index', 'Num_c', 'SMILES', 'CID', 'IUPACName'] + all_core_features].copy()

print("=== 机器学习就绪数据集 ===\n")
print(f"数据集形状: {df_ml_ready.shape}")
print(f"样本数: {len(df_ml_ready)}")
print(f"特征数: {len(all_core_features)}")
print(f"\n前5个样本预览:")
print(df_ml_ready.head())

=== 机器学习就绪数据集 ===

数据集形状: (235, 20)
样本数: 235
特征数: 15

前5个样本预览:
   Index  Num_c           SMILES     CID                 IUPACName  XLogP  \
0      1      4     C[N+](C)(C)C    6380        tetramethylazanium    0.3   
1      2      5    CC[N+](C)(C)C   33993   ethyl(trimethyl)azanium    0.6   
2      3      6   CCC[N+](C)(C)C   29385  trimethyl(propyl)azanium    1.2   
3      4      6   CC[N+](C)(C)CC  199517  diethyl(dimethyl)azanium    1.0   
4      5      7  CCCC[N+](C)(C)C   24399   butyl(trimethyl)azanium    1.5   

   Volume3D  MolecularWeight  Charge  EffectiveRotorCount3D  \
0      70.8            74.14       1                    0.0   
1      82.6            88.17       1                    1.0   
2      95.5           102.20       1                    2.0   
3      94.3           102.20       1                    2.0   
4     108.6           116.22       1                    3.0   

   RotatableBondCount  TPSA  Complexity  HeavyAtomCount  FeatureCationCount3D  \
0             

In [22]:
# 保存核心特征数据集
output_file = 'data/r4n_ml_features.csv'
df_ml_ready.to_csv(output_file, index=False)
print(f"\n✓ 核心特征数据集已保存至: {output_file}")

# 同时保存完整特征数据集
full_output_file = 'data/r4n_full_features.csv'
df_key_props.to_csv(full_output_file, index=False)
print(f"✓ 完整特征数据集已保存至: {full_output_file}")


✓ 核心特征数据集已保存至: data/r4n_ml_features.csv
✓ 完整特征数据集已保存至: data/r4n_full_features.csv


## 💡 总结与建议

### 关键发现:

1. **最重要的4个特征 (Tier 1)**:
   - `XLogP`: 疏水性参数,直接影响界面吸附倾向
   - `Volume3D`: 分子3D体积,决定表面覆盖和空间排布
   - `MolecularWeight`: 分子量,影响扩散和吸附动力学
   - `Charge`: 电荷(所有季铵离子均为+1)

2. **重要辅助特征 (Tier 2)**:
   - `EffectiveRotorCount3D` & `RotatableBondCount`: 柔性,影响表面构象适应
   - `TPSA`: 极性表面积,影响水溶性
   - `Complexity`: 分子复杂度

3. **3D结构特征 (Tier 3)**:
   - 立体四极矩 (X/Y/Z): 电荷分布的空间非对称性
   - 疏水/阳离子特征数: 功能位点信息

### 下一步建议:

1. **特征工程**:
   - 考虑添加衍生特征,如 `Volume3D/MolecularWeight` (密度指标)
   - 碳链长度 (`Num_c`) 与疏水性的交互项
   - 立体四极矩的总和/方向性

2. **筛选策略**:
   - 基于 `XLogP` 控制疏水性在合理范围
   - 基于 `Volume3D` 和 `EffectiveRotorCount3D` 优化表面适应性
   - 平衡分子量(不要太大影响扩散)

3. **建模方向**:
   - 使用核心特征集进行初步建模
   - 通过特征重要性分析验证特征选择
   - 如果需要可以加入完整特征集进行对比

In [23]:
# 创建特征重要性可视化摘要
print("=" * 80)
print("📊 特征筛选总结")
print("=" * 80)

print("\n✅ 已完成:")
print(f"  • 从PubChem获取了 235 个季铵阳离子的性质")
print(f"  • 识别了 18 个关键性质用于预测吸附能和水溶性")
print(f"  • 推荐了 15 个核心特征用于机器学习建模")

print("\n🎯 Tier 1 核心特征 (4个) - 直接影响吸附:")
tier1 = ['XLogP', 'Volume3D', 'MolecularWeight', 'Charge']
for feat in tier1:
    if feat in df_key_props.columns:
        cv = (df_key_props[feat].std() / df_key_props[feat].mean()) * 100
        print(f"  • {feat:25s} (CV: {cv:5.1f}%)")

print("\n📈 Tier 2 重要特征 (4个) - 构象和溶解性:")
tier2 = ['EffectiveRotorCount3D', 'RotatableBondCount', 'TPSA', 'Complexity']
for feat in tier2:
    if feat in df_key_props.columns:
        cv = (df_key_props[feat].std() / df_key_props[feat].mean()) * 100 if df_key_props[feat].mean() != 0 else 0
        print(f"  • {feat:25s} (CV: {cv:5.1f}%)")

print("\n📁 输出文件:")
print("  • r4n_ml_features.csv        - 15个核心特征 (推荐用于建模)")
print("  • r4n_full_features.csv      - 18个完整特征 (完整分析)")
print("  • r4n_c20_validated_pubchempy.csv - 所有PubChem数据")

print("\n💡 关键发现:")
print(f"  • XLogP 平均值: {df_key_props['XLogP'].mean():.2f} (疏水性较强)")
print(f"  • Volume3D 平均值: {df_key_props['Volume3D'].mean():.2f} Ų")
print(f"  • 有效转子数平均: {df_key_props['EffectiveRotorCount3D'].mean():.2f} (柔性分子)")
print(f"  • 所有离子电荷均为 +1")
print(f"  • TPSA = 0 (无极性表面,季铵阳离子特征)")

print("\n⚠️ 数据质量:")
print(f"  • 24个化合物缺少3D属性 (占 {24/235*100:.1f}%)")
print(f"  • 其余属性完整度: 100%")

print("\n" + "=" * 80)

📊 特征筛选总结

✅ 已完成:
  • 从PubChem获取了 235 个季铵阳离子的性质
  • 识别了 18 个关键性质用于预测吸附能和水溶性
  • 推荐了 15 个核心特征用于机器学习建模

🎯 Tier 1 核心特征 (4个) - 直接影响吸附:
  • XLogP                     (CV:  33.0%)
  • Volume3D                  (CV:  21.1%)
  • MolecularWeight           (CV:  21.7%)
  • Charge                    (CV:   0.0%)

📈 Tier 2 重要特征 (4个) - 构象和溶解性:
  • EffectiveRotorCount3D     (CV:  31.6%)
  • RotatableBondCount        (CV:  32.0%)
  • TPSA                      (CV:   0.0%)
  • Complexity                (CV:  28.6%)

📁 输出文件:
  • r4n_ml_features.csv        - 15个核心特征 (推荐用于建模)
  • r4n_full_features.csv      - 18个完整特征 (完整分析)
  • r4n_c20_validated_pubchempy.csv - 所有PubChem数据

💡 关键发现:
  • XLogP 平均值: 5.48 (疏水性较强)
  • Volume3D 平均值: 205.18 Ų
  • 有效转子数平均: 10.64 (柔性分子)
  • 所有离子电荷均为 +1
  • TPSA = 0 (无极性表面,季铵阳离子特征)

⚠️ 数据质量:
  • 24个化合物缺少3D属性 (占 10.2%)
  • 其余属性完整度: 100%



### 特征相关性分析

检查核心特征之间的相关性,避免冗余特征: